<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/week13/week13_avl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Árbol balanceado AVL

In [ ]:
%%writefile avl.h
#ifndef __AVL_H__
#define __AVL_H__

#include <functional>

using namespace std;

template<typename T, typename R=T, T NONE=0>
class AVLBST {
    struct Node;

    Node*           root;
    function<R(T)>  key;
    int             len;
public:
    AVLBST(function<R(T)> key=[](T a) { return a; })
        : key(key), root(nullptr), len(0) {}
    ~AVLBST()               { clear(root); }
    int height()            { return height(root); }
    int size()              { return len; }
    void clear()            { clear(root); len = 0; }

    void add(T elem)        { add(root, elem); }
    void remove(R val)      { remove(root, val); }

    T find(R val)           { return find(root, val); }
    T greaterThan(R val)    { return greaterThan(root, val); }

    void inOrder(function<void(T)> proc) { inOrder(root, proc); }

private:
    void    clear(Node*& node);
    int     height(Node* node);
    void    add(Node*& node, T elem);
    T       max(Node*& node);
    void    removeAux(Node*& node);
    void    remove(Node*& node, R val);
    T       find(Node* node, R val);
    T       greaterThan(Node* node, R val);
    void    inOrder(Node* node, function<void(T)> proc);

    void    updateHeight(Node* node);
    void    rotateLeft(Node*& node);
    void    rotateRight(Node*& node);
    void    balance(Node*& node);
};

#include "node.cpp"
#include "avl.cpp"
#include "balanceavl.cpp"

#endif

Writing avl.h


## Nodo del árbol
Se incluye la altura de un nodo como un atributo.

In [ ]:
%%writefile node.cpp
#include "avl.h"

template<typename T, typename R, T NONE>
struct AVLBST<T, R, NONE>::Node {
    T       element;
    Node*   left;
    Node*   right;
    int     height;

    Node(T element)
        : element(element), height(0),
          left(nullptr), right(nullptr) {}
};

Writing node.cpp


## Operaciones básicas
Se incluye la operación de obtener la altura de un nodo

In [ ]:
%%writefile avl.cpp
#include "avl.h"

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::clear(Node*& node) {
    if (node != nullptr) {
        clear(node->left);
        clear(node->right);
        delete node;
        node = nullptr;
    }
}

template<typename T, typename R, T NONE>
int AVLBST<T, R, NONE>::height(Node* node) {
    return node == nullptr? -1 : node->height;
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::add(Node*& node, T elem) {
    if (node == nullptr) {
        node = new Node(elem);
        ++len;
    } else {
        if (key(elem) < key(node->element)) {
            add(node->left, elem);
        } else {
            add(node->right, elem);
        }
        balance(node);
    }
}

template<typename T, typename R, T NONE>
T AVLBST<T, R, NONE>::max(Node*& node) {
    if (node->right == nullptr) {
        auto aux = node;
        node = node->left;
        auto auxElem = aux->elem;
        delete aux;
        return auxElem;
    } else {
        auto auxElem = max(node->right);
        balance(node);
        return auxElem;
    }
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::removeAux(Node*& node) {
    if (node->left == nullptr) {
        auto aux = node;
        node = node->right;
        delete aux;
    } else {
        node->elem = max(node->left);
    }
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::remove(Node*& node, R val) {
    if (node != nullptr) {
        if (val == key(node->elem)) {
            removeAux(node);
        } else {
            if (val < key(node->elem)) {
                remove(node->left, val);
            } else {
                remove(node->right, val);
            }
            balance(node);
        }
    }
}

template<typename T, typename R, T NONE>
T AVLBST<T, R, NONE>::find(Node* node, R val) {
    if (node == nullptr) {
        return NONE;
    } else if (val == key(node->element)) {
        return node->element;
    } else if (val < key(node->element)) {
        return find(node->left, val);
    } else {
        return find(node->right, val);
    }
}

template<typename T, typename R, T NONE>
T AVLBST<T, R, NONE>::greaterThan(Node* node, R val) {
    if (node == nullptr) {
        return NONE;
    } else if (val < key(node->element)) {
        return node->element;
    }
    return greaterThan(node->right, val);
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::inOrder(Node* node,
                                 function<void(T)> proc) {
    if (node != nullptr) {
        inOrder(node->left, proc);
        proc(node->element);
        inOrder(node->right, proc);
    }
}

Overwriting avl.cpp


## Operaciones para el balanceo

In [ ]:
%%writefile balanceavl.cpp
#include "avl.h"

#define max(a, b) (a > b? a : b)

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::updateHeight(Node* node) {
    if (node != nullptr) {
        int hl = height(node->left);
        int hr = height(node->right);

        node->height = max(hl, hr) + 1;
    }
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::rotateLeft(Node*& node) {
    Node* aux = node->right;
    node->right = aux->left;
    updateHeight(node);
    aux->left = node;
    updateHeight(aux);
    node = aux;
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::rotateRight(Node*& node) {
    Node* aux = node->left;
    node->left = aux->right;
    updateHeight(node);
    aux->right = node;
    updateHeight(aux);
    node = aux;
}

template<typename T, typename R, T NONE>
void AVLBST<T, R, NONE>::balance(Node*& node) {
    int hl = height(node->left);
    int hr = height(node->right);

    if (hr - hl < -1) {
        hl = height(node->left->left);
        hr = height(node->left->right);
        if (hr > hl) {
            rotateLeft(node->left);
        }
        rotateRight(node);
    } else if (hr - hl > 1) {
        hl = height(node->right->left);
        hr = height(node->right->right);
        if (hl > hr) {
            rotateRight(node->right);
        }
        rotateLeft(node);
    } else {
        updateHeight(node);
    }
}

Writing balanceavl.cpp


## Un pequeño ejemplo
Un árbol avl de elementos enteros. Nótese que para un millón de elementos su altura es 19, pero para elementos aleatorios la altura podría variar entre 19 y 22 o 23.

In [ ]:
%%writefile avlTest.cpp
#include <iostream>
#include <string>
#include "avl.h"

using namespace std;

int main() {
    AVLBST<string*, string, nullptr>* avl = 
        new AVLBST<string*, string, nullptr>([](string* a){return *a;});

    for (int i = 0; i < 1000000; ++i) {
        avl->add(new string(to_string(i + 1)));
    }

    cout << avl->height() << endl;

    delete avl;
    return 0;
}

Overwriting avlTest.cpp


In [ ]:
%%script bash
g++ -std=c++17 avlTest.cpp && ./a.out

22


## Un ejemplo más complejo
Creamos una clase Alumno para nuestro ejemplo.

In [ ]:
%%writefile alumno.h
#ifndef __ALUMNO_H__
#define __ALUMNO_H__

#include <string>

class Alumno {
    long long   dni;
    std::string nombre;
    std::string apellido;

public:
    Alumno(long long dni, std::string nombre, std::string apellido)
        : dni(dni), nombre(nombre), apellido(apellido) {}
    long long   getDni()        { return dni; }
    std::string getNombre()     { return nombre; }
    std::string getApellido()   { return apellido; }

    void setDni(long long dni)              { this->dni = dni; }
    void setNombre(std::string nombre)      { this->nombre = nombre; }
    void setApellido(std::string apellido)  { this->apellido = apellido; }
};

#endif

Overwriting alumno.h


Crearemos un árbol por cada atributo, para demostrar el uso de los árboles.

In [ ]:
%%writefile avlAlumnoTest.cpp
#include <iostream>
#include <iomanip>
#include <vector>
#include "alumno.h"
#include "avl.h"

using namespace std;

int main() {
    auto k1 = [](Alumno* a) { return a->getDni(); };
    auto k2 = [](Alumno* a) { return a->getNombre(); };
    auto k3 = [](Alumno* a) { return a->getApellido(); };

    AVLBST<Alumno*, long long, nullptr>* idxDni = 
        new AVLBST<Alumno*, long long, nullptr>(k1);
    AVLBST<Alumno*, string, nullptr>*    idxNom =
        new AVLBST<Alumno*, string, nullptr>(k2);
    AVLBST<Alumno*, string, nullptr>*    idxApe =
        new AVLBST<Alumno*, string, nullptr>(k3);

    vector<Alumno*> alumnos;
    alumnos.push_back(new Alumno(21, "Juan", "Perez"));
    alumnos.push_back(new Alumno(54, "Rosa", "Jimenez"));
    alumnos.push_back(new Alumno(72, "Maria", "Gonzalez"));
    alumnos.push_back(new Alumno(11, "Ramona", "Cueva"));
    alumnos.push_back(new Alumno(34, "José", "Ugarte"));
    alumnos.push_back(new Alumno(98, "Ricardo", "Shiguihara"));
    alumnos.push_back(new Alumno(37, "Manuel", "Villalta"));
    alumnos.push_back(new Alumno(17, "Rodolfa", "Rodriguez"));
    alumnos.push_back(new Alumno(41, "Patricia", "Smith"));
    alumnos.push_back(new Alumno(19, "Aquiles", "Zeballos"));
    alumnos.push_back(new Alumno(25, "Carlos", "Wayne"));

    for (auto al : alumnos) {
        idxDni->add(al);
        idxNom->add(al);
        idxApe->add(al);
    }

    auto print = [](Alumno* a) {
        if (a == nullptr) return;
        cout << setw(6) << a->getDni()
            << setw(12) << a->getNombre()
            << setw(12) << a->getApellido() << endl;
    };

    idxDni->inOrder(print); cout << "-----\n";
    idxNom->inOrder(print); cout << "-----\n";
    idxApe->inOrder(print); cout << "-----\n";

    print(idxDni->greaterThan(11));
    print(idxNom->find("Ricardo"));
    print(idxApe->find("Smith"));

    delete idxDni;
    delete idxApe;
    delete idxNom;
    return 0;
}

Writing avlAlumnoTest.cpp


In [ ]:
%%script bash
g++ -std=c++17 avlAlumnoTest.cpp && ./a.out

    11      Ramona       Cueva
    17     Rodolfa   Rodriguez
    19     Aquiles    Zeballos
    21        Juan       Perez
    25      Carlos       Wayne
    34       José      Ugarte
    37      Manuel    Villalta
    41    Patricia       Smith
    54        Rosa     Jimenez
    72       Maria    Gonzalez
    98     Ricardo  Shiguihara
-----
    19     Aquiles    Zeballos
    25      Carlos       Wayne
    34       José      Ugarte
    21        Juan       Perez
    37      Manuel    Villalta
    72       Maria    Gonzalez
    41    Patricia       Smith
    11      Ramona       Cueva
    98     Ricardo  Shiguihara
    17     Rodolfa   Rodriguez
    54        Rosa     Jimenez
-----
    11      Ramona       Cueva
    72       Maria    Gonzalez
    54        Rosa     Jimenez
    21        Juan       Perez
    17     Rodolfa   Rodriguez
    98     Ricardo  Shiguihara
    41    Patricia       Smith
    34       José      Ugarte
    37      Manuel    Villalta
    25      Carlos       Wayne